# Load Libraries

In [1]:
# numpy and pandas
import numpy as np
import pandas as pd
import math

# Generic
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Images
from PIL import Image
from skimage.transform import resize
import cv2
# import talos as ta

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, plot_confusion_matrix, roc_curve


# Tensorflow
import tensorflow as tf
# import tensorflow.compat.v1 as tf

# Keras
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.utils import print_summary
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau, Callback
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras import metrics
from matplotlib.ticker import MaxNLocator

Using TensorFlow backend.


# Global Paths

In [2]:
path_root = '/home/ygala/TFM_UOC/'
path_images = os.path.join(path_root, 'data', 'covid_images')
path_results = os.path.join(os.getcwd(),'final_model_results')

# Global Parameters

In [3]:
perc_split = [0.7, 0.15, 0.15]
input_shape = (224, 224, 3)
seed = 14
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Load Classes and Functions

### CheXnet

In [4]:
# chexNet weights
chexnet_weights = '/home/ygala/TFM_UOC/scripts/chexnet/best_weights.h5'

def chexnet_preprocess_input(value):
    return preprocess_input(value)


def get_chexnet_model():
    input_shape = (224, 224, 3)
    img_input = Input(shape=input_shape)
    base_weights = 'imagenet'

    # create the base pre-trained model
    base_model = DenseNet121(
        include_top=False,
        input_tensor=img_input,
        input_shape=input_shape,
        weights=base_weights,
        pooling='avg'
    )

    x = base_model.output
    # add a logistic layer -- let's say we have 14 classes
    predictions = Dense(
        14,
        activation='sigmoid',
        name='predictions')(x)

    # this is the model we will use
    model = Model(
        inputs=img_input,
        outputs=predictions,
    )

    # load chexnet weights
    model.load_weights(chexnet_weights)

    # return model
    return base_model, model

### Auxiliary functions

In [5]:
def get_class_weight(csv_file_path, target_class):
    df = pd.read_csv(csv_file_path, sep=';')
    total_counts = df.shape[0]
    class_weight = []

    ratio_pos = df.loc[(df[target_class] == 'Y')].shape[0] / total_counts
    ratio_neg = df.loc[(df[target_class] == 'N')].shape[0] / total_counts
    class_weight = np.array((ratio_pos, ratio_neg))
        
    return class_weight

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc


def print_confidence_intervals(statistics):
    df = pd.DataFrame(columns=["Mean AUC (CI 5%-95%)"])
    mean = statistics.mean()
    max_ = np.quantile(statistics, .95)
    min_ = np.quantile(statistics, .05)
    df.loc["Exitus"] = ["%.2f (%.2f-%.2f)" % (mean, min_, max_)]
    return df


def get_model(learning_rate):
    # get base model, model
    base_model, chexnet_model = get_chexnet_model()

    x = base_model.output
    
    # Regularization layer
    x = BatchNormalization()(x)
    
    # Dense layer
    x = Dense(128, 
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l1_l2(0.5, 0.0001))(x)
    
    # Regularization layer
    x = BatchNormalization()(x)

    
    # add a logistic layer -- let's say we have 6 classes
    predictions = Dense(
        1,
        activation='sigmoid')(x)

    # this is the model we will use
    model = Model(
        inputs=base_model.input,
        outputs=predictions,
    )

    # first: train only the top layers (which were randomly initialized)

    for layer in base_model.layers:
        layer.trainable = False

    # initiate an Adam optimizer
    opt = Adam(
        lr=learning_rate,
        beta_1=0.9,
        beta_2=0.999,
        decay=0.0,
        amsgrad=False
    )

    # Let's train the model using Adam
    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=[metrics.BinaryAccuracy(name = "acc"),
                metrics.AUC(name = "auc")])

    return base_model, model


class print_learning_rate(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        print(f'Learning rate = {K.eval(lr):.5f}')



### Print functions

In [6]:
def grafica_entrenamiento(tr_auc, val_auc, tr_loss, val_loss, best_i,
                          figsize=(10,5), path_results = None):
    plt.figure(figsize=figsize)
    ax = plt.subplot(1,2,1)
    plt.plot(1+np.arange(len(tr_loss)), np.array(tr_loss))
    plt.plot(1+np.arange(len(val_loss)), np.array(val_loss))
    plt.plot(1+best_i, val_loss[best_i], 'or')
    plt.title('loss del modelo', fontsize=18)
    plt.ylabel('loss', fontsize=12)
    plt.xlabel('época', fontsize=18)        
    plt.legend(['entrenamiento', 'validación'], loc='upper left')
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    
    plt.subplot(1,2,2)
    
    plt.plot(1+np.arange(len(tr_auc)),  np.array(tr_auc))
    plt.plot(1+np.arange(len(val_auc)), np.array(val_auc))
    plt.plot(1+best_i, val_auc[best_i], 'or')
    plt.title('AUC', fontsize=18)
    plt.ylabel('AUC', fontsize=12)
    plt.xlabel('época', fontsize=18)    
    plt.legend(['entrenamiento', 'validación'], loc='upper left')
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    if (path_results != None):
        plt.savefig(os.path.join(path_results, 'auc_loss.png'))
    plt.show()
    
class TrainingPlot(Callback):
    
    # This function is called when the training begins
    def on_train_begin(self, logs={}):
        # Initialize the lists for holding the logs, losses and accuracies
        self.losses = []
        self.auc = []
        self.val_losses = []
        self.val_auc = []
        self.logs = []
    
    # This function is called at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):       
       
        
        # Before plotting ensure at least 10 epochs have passed
        if epoch > 20:
             # Append the logs, losses and accuracies to the lists
            self.logs.append(logs)        
            self.auc.append(logs.get('auc'))        
            self.val_auc.append(logs.get('val_auc'))
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            best_i = np.argmax(self.val_auc)
            grafica_entrenamiento(self.auc, self.val_auc, self.losses, self.val_losses, best_i)

plot_losses = TrainingPlot()

### Heatmaps Functions

In [8]:
def show_heatmap(model, im, es_maligna, predictions):
    


    imag = np.reshape(im, (1, im.shape[0], im.shape[1], im.shape[2]))
        
    # This is the "benign" entry in the prediction vector
    output = model.output[0, 0]
    
    # The is the output feature map of the last convolutional layer
    last_conv_layer = model.get_layer('bn')
    
    # This is the gradient of the "benign" class with regard to
    # the output feature map of last convolutional layer
    grads = K.gradients(output, last_conv_layer.output)[0]
    
    
    # This function allows us to access the values of the quantities we just defined:
    # `pooled_grads` and the output feature map of the last convolutional layer
    # given a sample image
    iterate = K.function([model.input], [last_conv_layer.output, grads])
    
    # These are the values of these two quantities, as Numpy arrays,
    # given our sample image
    output, grads_val = iterate(imag)
    conv_layer_output_value, pooled_grads_value = output[0, :], grads_val[0, :, :, :]   
    
   
    
      
    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    weights = np.mean(pooled_grads_value, axis=(0, 1))
    cam = np.dot(conv_layer_output_value, weights)
    heatmap = np.maximum(cam, 0)
    heatmap /= np.max(heatmap)
    plt.matshow(heatmap)
    plt.show()
    
    # load the original image
    img = imag[0]
    
    # Process CAM
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()  


    
    # We resize the heatmap to have the same size as the original image
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # We convert the heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    
    # We apply the heatmap to the original image
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    superimposed_img = heatmap * 0.8 / 255 + 0.8*img

    
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(img, vmin=0, vmax=1)
    plt.subplot(1,3,2)
    plt.imshow(heatmap, vmin=0, vmax=1)
    plt.colorbar()
    plt.subplot(1,3,3)
    plt.imshow(img,
                       cmap='gray')
    plt.imshow(cam, cmap='jet', alpha=min(0.5, predictions[0]))
    plt.colorbar()
    plt.show()
    print("- Probabilidad de Exitus:", predictions[0])
    print("-", "Clase real:", "No sobrevive" if es_maligna else "Sobrevive")
    print("\n\n\n")
    return heatmap, superimposed_img

# Load Data

## Read data

In [9]:
case_features = pd.read_csv(os.path.join(path_images, 'clean_data_1_filtered_2.csv'), sep=';');
case_features

,patientid,filename,index,SpecificCharacterSet,SOPClassUID,SOPInstanceUID,StudyDate,StudyTime,AccessionNumber,Modality,...,BitsStored,HighBit,PixelRepresentation,LossyImageCompression,LossyImageCompressionMethod,PixelData,image_date,survival,fec_ing,days
0,14,183271.png,one,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003240757170156,20200324,75717.0,ACC00005178,CR,...,12,11,0,0,NaN,Array of 5987418 elements,2020-03-24,Y,2020-03-21,3
1,14,183272.png,one,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003240804560406,20200324,75717.0,ACC00005178,CR,...,12,11,0,0,NaN,Array of 5877340 elements,2020-03-24,Y,2020-03-21,3
2,18,330384.png,one,NaN,Computed Radiography Image Storage,1.3.51.0.7.14038266962.53702.43073.38236.31465...,20200327,73607.0,ACC00006084,CR,...,12,11,0,0,NaN,Array of 7269056 elements,2020-03-27,Y,2020-03-24,3
3,24,480742.png,one,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003230325420000,20200323,32542.0,ACC00005564,CR,...,12,11,0,0,NaN,Array of 5605756 elements,2020-03-23,Y,2020-03-20,3
4,25,490931.png,one,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003190646480968,20200319,64648.0,ACC00004612,CR,...,12,11,0,0,NaN,Array of 6076912 elements,2020-03-19,Y,2020-03-17,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,2257,451177.png,one,NaN,Computed Radiography Image Storage,1.3.51.0.7.1865235605.2972.30027.36001.12312.3...,20200408,190450.0,ACC00009861,CR,...,12,11,0,0,NaN,Array of 4741754 elements,2020-04-08,N,2020-04-06,2
1274,2269,457682.png,one,NaN,Computed Radiography Image Storage,1.3.51.0.7.13791659901.14351.1090.48094.31869....,20200414,171700.0,ACC00009957,CR,...,12,11,0,0,NaN,Array of 5894224 elements,2020-04-14,Y,2020-04-10,4
1275,2276,459945.png,one,ISO_IR 100,Computed Radiography Image Storage,1.3.51.0.7.3052672552.30103.8777.44918.7380.44...,20200413,100546.0,ACC00010030,CR,...,12,11,0,0,NaN,Array of 6833352 elements,2020-04-13,Y,2020-04-10,3
1276,2280,461518.png,one,NaN,Computed Radiography Image Storage,1.3.51.0.7.11525005549.51456.28486.35726.61854...,20200415,103742.0,ACC00010093,CR,...,12,11,0,0,NaN,Array of 5694582 elements,2020-04-15,Y,2020-04-11,4


## Split data

In [10]:
# target_class = 'survival'
# np.random.seed(seed)

# ### Train and val
# # Split patient ids
# positive = case_features[case_features.survival.values == 'Y']
# patient_ids = positive.patientid.unique()
# original_length = len(patient_ids)
# train_ids = np.random.choice(patient_ids, size = math.floor(perc_split[0]*original_length), replace = False)
# patient_ids = patient_ids[~np.isin(patient_ids, train_ids)];
# val_ids = np.random.choice(patient_ids, size = math.floor(perc_split[1]*original_length), replace = False)
# test_ids = patient_ids[~np.isin(patient_ids, val_ids)];

# negative = case_features[case_features.survival.values == 'N']
# patient_ids = negative.patientid.unique()
# original_length = len(patient_ids)
# train_ids = np.append(train_ids, np.random.choice(patient_ids, size = math.floor(perc_split[0]*original_length), replace = False))
# patient_ids = patient_ids[~np.isin(patient_ids, train_ids)];
# val_ids = np.append(val_ids, np.random.choice(patient_ids, size = math.floor(perc_split[1]*original_length), replace = False))
# test_ids = np.append(test_ids, patient_ids[~np.isin(patient_ids, val_ids)]);

# # Split dataset based on patient ids
# case_features_train = case_features[case_features.patientid.isin(train_ids)]
# case_features_val = case_features[case_features.patientid.isin(val_ids)]
# case_features_test = case_features[case_features.patientid.isin(test_ids)]

# # Selección del patrón de datos X y del target y
# ytrain = case_features_train[target_class]
# del case_features_train[target_class]
# Xtrain = case_features_train

# yval = case_features_val[target_class]
# del case_features_val[target_class]
# Xval = case_features_val

# ytest = case_features_test[target_class]
# del case_features_test[target_class]
# Xtest = case_features_test

In [11]:
# print(len(case_features_train[ytrain == "Y"].patientid.unique()))
# print(len(case_features_train[ytrain == "N"].patientid.unique()))
# print(len(case_features_val[yval == "Y"].patientid.unique()))
# print(len(case_features_val[yval == "N"].patientid.unique()))
# print(len(case_features_test[ytest == "Y"].patientid.unique()))
# print(len(case_features_test[ytest == "N"].patientid.unique()))

In [12]:
# print(case_features.shape)
# print(case_features_train.shape)
# print(case_features_val.shape)
# print(case_features_test.shape)

In [13]:
# print(ytrain.value_counts())
# print(yval.value_counts())
# print(ytest.value_counts())

## Read images

### Train

In [14]:
# X_train = []
# for i in range(Xtrain.shape[0]):
#     # print('%0.2f%%' % float(100*i/Xtrain.shape[0]))
#     image_path = os.path.join(path_images, 'processed', Xtrain.iloc[i].filename)
#     imagen = Image.open(image_path)
#     imagen = np.asarray(imagen.convert("RGB"))
#     imagen = resize(imagen,  input_shape)
#     X_train.append(imagen)

# X_train = np.stack(X_train, axis = 0)
# np.save(os.path.join(path_images, 'X_train.npy'), X_train)
# print(X_train.shape)

### Validation

In [15]:
# X_val = []
# for i in range(Xval.shape[0]):
#     # print('%0.2f%%' % float(100*i/Xval.shape[0]))
#     image_path = os.path.join(path_images, 'processed', Xval.iloc[i].filename)
#     imagen = Image.open(image_path)
#     imagen = np.asarray(imagen.convert("RGB"))
#     imagen = resize(imagen,  input_shape)
#     X_val.append(imagen)

# X_val = np.stack(X_val, axis = 0)
# np.save(os.path.join(path_images, 'X_val.npy'), X_val)
# print(X_val.shape)

### Test

In [16]:
# X_test = []
# for i in range(Xtest.shape[0]):
#     # print('%0.2f%%' % float(100*i/Xtest.shape[0]))
#     image_path = os.path.join(path_images, 'processed', Xtest.iloc[i].filename)
#     imagen = Image.open(image_path)
#     imagen = np.asarray(imagen.convert("RGB"))
#     imagen = resize(imagen,  input_shape)
#     X_test.append(imagen)

# X_test = np.stack(X_test, axis = 0)
# np.save(os.path.join(path_images, 'X_test.npy'), X_test)
# print(X_test.shape)

## Set target

### Train

In [17]:
# ytrain[ytrain=='Y'] = 0
# ytrain[ytrain=='N'] = 1
# y_train = np.array(ytrain, dtype = np.int64)
# y_train.shape
# np.save(os.path.join(path_images, 'y_train.npy'), y_train)

### Validation

In [18]:
# yval[yval=='Y'] = 0
# yval[yval=='N'] = 1
# y_val = np.array(yval, dtype = np.int64)
# y_val.shape
# np.save(os.path.join(path_images, 'y_val.npy'), y_val)

### Test

In [19]:
# ytest[ytest=='Y'] = 0
# ytest[ytest=='N'] = 1
# y_test = np.array(ytest, dtype = np.int64)
# y_test.shape
# np.save(os.path.join(path_images, 'y_test.npy'), y_test)

In [20]:
X_train = np.load(os.path.join(path_images, 'X_train.npy'))
X_val = np.load(os.path.join(path_images, 'X_val.npy'))
X_test = np.load(os.path.join(path_images, 'X_test.npy'))
y_train = np.load(os.path.join(path_images, 'y_train.npy'))
y_val = np.load(os.path.join(path_images, 'y_val.npy'))
y_test = np.load(os.path.join(path_images, 'y_test.npy'))

## Set class weights

In [21]:
# train
ratio_pos = np.count_nonzero(y_train == 0) / len(y_train)
ratio_neg = np.count_nonzero(y_train == 1) / len(y_train)
class_weight_train = np.array((ratio_pos, ratio_neg))

# val
ratio_pos = np.count_nonzero(y_val == 0) / len(y_val)
ratio_neg = np.count_nonzero(y_val == 1) / len(y_val)
class_weight_val = np.array((ratio_pos, ratio_neg))

# test
ratio_pos = np.count_nonzero(y_test == 0) / len(y_test)
ratio_neg = np.count_nonzero(y_test == 1) / len(y_test)
class_weight_test= np.array((ratio_pos, ratio_neg))

# Print
print(class_weight_train)
print(class_weight_val)
print(class_weight_test)


[0.84575835 0.15424165]
[0.85772358 0.14227642]
[0.8503937 0.1496063]


# Data Augmentation

In [22]:
datagen = ImageDataGenerator(featurewise_center=True, 
                             featurewise_std_normalization=True,
                             brightness_range = (0.25, 0.75))
datagen.fit(X_train)

# Model Definition

## Callbacks

### Checkpoints

In [23]:
save_dir = os.path.join(
    os.getcwd(),
    '../saved_models'
)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

# This callback saves the weights of the model after each epoch
checkpoint = ModelCheckpoint(
    '../saved_models/weights.epoch_{epoch:02d}.hdf5',
    monitor='val_loss', 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto',
    verbose=1
)

# This callback writes logs for TensorBoard
tensorboard = TensorBoard(
    log_dir='./Graph', 
    histogram_freq=0,  
    write_graph=True
)



early_stopping = EarlyStopping(monitor='val_auc', patience=400, verbose=1, mode='max', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100, mode='min')



callbacks_list = [early_stopping]


# GRID

In [24]:
epochs = 1000
### Hyperparameters
batch_size_values = [32, 64, 128, 512]
learning_rate_values = [0.0001, 0.001, 0.01, 1]
neurons_values = [64, 128, 256, 512]
l2_values = [0.001, 0.01, 0.1, 0.5]

In [ ]:
grid_results = pd.DataFrame()
for batch_size in batch_size_values:
    for learning_rate in learning_rate_values:
        for neurons in neurons_values:
            for l2 in l2_values:
                
                ### Print log
                print ("Start of batch_size = %d - learning_rate = %f - neurons = %d - l2 = %f" % 
                       (batch_size, learning_rate, neurons, l2))
                
                ### Standard model
                
                # Create model
                base_model, model = get_model(learning_rate, neurons, l2)
                
                # Train
                history = model.fit(X_train,y_train,
                          validation_data=(X_val, y_val),
                          batch_size=batch_size, 
                          nb_epoch=epochs,
                          class_weight=class_weight_train,
                          callbacks = callbacks_list,
                          verbose=0)
                
                # Predict
                pred_train = model.predict(X_train)
                pred_val = model.predict(X_val)
                pred_test = model.predict(X_test)
                
                # Get metrics
                stopped_epoch = early_stopping.stopped_epoch
                auc_train = roc_auc_score(y_true = y_train, y_score = pred_train)
                auc_val = roc_auc_score(y_true = y_val, y_score = pred_val)
                auc_test = roc_auc_score(y_true = y_test, y_score = pred_test)
                
                # Save results
                res = pd.DataFrame([batch_size, learning_rate, neurons, l2, 
                                    stopped_epoch, auc_train, auc_val, auc_test])
                grid_results = pd.concat([grid_results, res], axis=1)
                
                # Free memory                
                del base_model, model
                
                
                ### Augmented model
                
                # Create model
                base_model_augmented, model_augmented = get_model(learning_rate, neurons, l2)
                
                # Train
                history = model_augmented.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size, seed=seed),
                     validation_data=(X_val, y_val),
                     steps_per_epoch=len(X_train) / batch_size, 
                     epochs=epochs,
                     class_weight=class_weight_train,
                     callbacks = callbacks_list,       
                     verbose=0)
                
                # Predict
                pred_train = model_augmented.predict(X_train)
                pred_val = model_augmented.predict(X_val)
                pred_test = model_augmented.predict(X_test)
                
                # Get metrics
                stopped_epoch = early_stopping.stopped_epoch
                auc_train = roc_auc_score(y_true = y_train, y_score = pred_train)
                auc_val = roc_auc_score(y_true = y_val, y_score = pred_val)
                auc_test = roc_auc_score(y_true = y_test, y_score = pred_test)
                
                # Save results
                res = pd.DataFrame([batch_size, learning_rate, neurons, l2, 
                                    stopped_epoch, auc_train, auc_val, auc_test])
                grid_results = pd.concat([grid_results, res], axis=1)      
                
                # Free memory   
                del base_model_augmented, model_augmented
                
                
                # Save intermediate results
                inner_res = grid_results.copy()
                inner_res.index = ['batch_size', 'learning_rate', 'neurons', 'l2', 
                                      'stopped_epoch', 'auc_train', 'auc_val', 'auc_test']
                inner_res.T.to_csv(os.path.join(path_results,'grid_results.csv')) 

# Save final results
grid_results.index = ['batch_size', 'learning_rate', 'neurons', 'l2', 
                      'stopped_epoch', 'auc_train', 'auc_val', 'auc_test']
grid_results.T.to_csv(os.path.join(path_results,'grid_results.csv'))      

Start of batch_size = 32 - learning_rate = 0.000100 - neurons = 64 - l2 = 0.001000


/usr/local/miniconda/python-3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Restoring model weights from the end of the best epoch
Epoch 00574: early stopping
Restoring model weights from the end of the best epoch
Epoch 00564: early stopping
Start of batch_size = 32 - learning_rate = 0.000100 - neurons = 64 - l2 = 0.010000


/usr/local/miniconda/python-3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Restoring model weights from the end of the best epoch
Epoch 00450: early stopping
Restoring model weights from the end of the best epoch
Epoch 00529: early stopping
Start of batch_size = 32 - learning_rate = 0.000100 - neurons = 64 - l2 = 0.100000


/usr/local/miniconda/python-3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Restoring model weights from the end of the best epoch
Epoch 00434: early stopping
Restoring model weights from the end of the best epoch
Epoch 00493: early stopping
Start of batch_size = 32 - learning_rate = 0.000100 - neurons = 64 - l2 = 0.500000


/usr/local/miniconda/python-3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Restoring model weights from the end of the best epoch
Epoch 00467: early stopping
